# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
Project submission,   2017/05/29

---


In [1]:
#importing packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
from scipy import stats
import math
%matplotlib inline

Once Packages are imported, the functions to be used are defined

In [2]:

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image
            
def draw_lines_ik(img, lines, color=[255, 0, 0], thickness=2):  # Function for drawing processed lines in images
    
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    mid = img.shape[1]/2  # Get image medium point
    r_x = []              # Initialize lists
    r_y = []
    l_x = []
    l_y = []
    
    # Process the lines data
    for line in lines:
        for x1,y1,x2,y2 in line:
            m,b = np.polyfit((x1,x2),(y1,y2),1)  # First grade polyfit for slope
            if 0.2 < np.absolute(m) < 1:         # Check if slope is between desirable values
                if m > 0 and x1 > mid:           # Check if fits criteria for right line
                    r_x.append(x1)
                    r_x.append(x2)
                    r_y.append(y1)
                    r_y.append(y2)
                elif m < 0 and x2 < mid:        # Check if fits criteria for left line
                    l_x.append(x1)
                    l_x.append(x2)
                    l_y.append(y1)
                    l_y.append(y2)
                    
    # Draw right line 
    q,s = np.polyfit(r_x,r_y,1)                 # First grade polyfit between right lane data points
    r_index = r_y.index(max(r_y))               # Start line in the max y position
    r_y1 = img.shape[0] 
    r_x1 = int((r_y1-s)/q)
    r_y2 = 330                                  # End line where mask ends
    r_x2 = int((r_y2-s)/q)
    cv2.line(line_img, (r_x1, r_y1), (r_x2, r_y2), color, thickness) # Draw line
    
    # Draw left line 
    q,s = np.polyfit(l_x,l_y,1)
    l_y1 = r_y1
    l_x1 = int((l_y1-s)/q)
    l_y2 = 330
    l_x2 = int((l_y2-s)/q)
    cv2.line(line_img, (l_x1, l_y1), (l_x2, l_y2), color, thickness)
    
    return line_img

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

def hough_lines_array(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    return lines

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## PROCESS PIPELINE

In [3]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    imshape = image.shape
    gray = grayscale(image)
    blur = gaussian_blur(gray, 7)
    canny_f = canny(blur, 40, 80)
    imshape = canny_f.shape
    vertices = np.array([[(140,imshape[0]),(430, 330), (540, 330), (imshape[1]-50,imshape[0])]], dtype=np.int32)
    mask = region_of_interest(canny_f, vertices)
    hough_array = hough_lines_array(mask, 1, np.pi/180, 4, 10, 5)
    lines_img = draw_lines_ik(image, hough_array, color=[255, 0, 0], thickness=10)
    weight = weighted_img(lines_img, image, α=0.8, β=1., λ=0.)

    return weight

## IMAGES

In [4]:
import os
os.listdir("test_images/")
import scipy.misc

for filename in os.listdir("test_images/"):
    img = mpimg.imread("test_images/" + filename)
    weight = process_image(img)
    scipy.misc.imsave("test_images_out/" + filename, weight)
    # Just for test purpouses
    #plt.imshow(weight) 

## VIDEOS

In [5]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

First video

In [6]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|█████████▉| 221/222 [00:02<00:00, 80.52it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

CPU times: user 2.66 s, sys: 637 ms, total: 3.29 s
Wall time: 3.1 s


In [7]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|█████████▉| 681/682 [00:09<00:00, 75.19it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

CPU times: user 8.84 s, sys: 2 s, total: 10.8 s
Wall time: 9.45 s


In [8]:
challenge_output = 'test_videos_output/challenge.mp4'
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/challenge.mp4
[MoviePy] Writing video test_videos_output/challenge.mp4


100%|██████████| 251/251 [00:11<00:00, 22.71it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge.mp4 

CPU times: user 10.1 s, sys: 1.67 s, total: 11.8 s
Wall time: 11.8 s
